<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
! pip -q install transformers
! pip -q install parsivar

In [3]:
import torch
from __future__ import unicode_literals
# from hazm import *
from parsivar import *
from pprint import pprint
import torch
import numpy as np
import pandas as pd
from datetime import datetime
from scipy import spatial
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForTokenClassification
parsivar_tokenizer = Tokenizer()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-uncased")
# tokenizer = AutoTokenizer.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model = AutoModelForTokenClassification.from_pretrained("HooshvareLab/bert-base-parsbert-ner-uncased")
model.eval()

# **Tokenize**

### Example

In [22]:
tweet = """ بابا بیخیال :)) همون سال اتلتیکو با گل افساید به بایرن رفت فینال.

مشکل داوری نداشتیم اون بازی که میگی.
فصل قبلش هم ۴ ۰ با هاینکس زدیم شما رو
"""

In [23]:
input_sentence = tokenizer.tokenize(tweet)
print(input_sentence)

['بابا', 'بیخیال', ':', ')', ')', 'همون', 'سال', 'اتلتیکو', 'با', 'گل', 'افساید', 'به', 'بایرن', 'رفت', 'فینال', '.', 'مشکل', 'داوری', 'نداشتیم', 'اون', 'بازی', 'که', 'میگی', '.', 'فصل', 'قبلش', 'هم', '۴', '۰', 'با', 'هاینکس', 'زدیم', 'شما', 'رو']


### Tokenize All tweets

In [ ]:
! wget "https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/extracted_data.txt"

In [31]:
! head extracted_data.txt

سلام بر آن بانوی بزرگوار س
صبح_بخیر 

**************

خدایا به ما توفیق فروکردن کلید آنخ ماعو از فیلم یوزارسیف رو به تحال این شیخ کذاب بده.

**************

خدایا به ما توفیق درآوردن ش ل و ا ر  از پ ا ی این شیخ کذاب، بده.


In [36]:
tweets_str = ""
with open("extracted_data.txt") as file:
  for line in file.readlines():
    tweets_str += line
tweets_str[:20]

'سلام بر آن بانوی بزر'

In [37]:
tweets = tweets_str.split("\n**************\n")
tweets[0]

'سلام بر آن بانوی بزرگوار س\nصبح_بخیر \n'

In [38]:
for tweet in tweets:
  print(tweet)
  break

سلام بر آن بانوی بزرگوار س
صبح_بخیر 



# **Model**

In [18]:
from transformers import pipeline

In [25]:
pars_ner = pipeline('ner', model=model, tokenizer=tokenizer)
pars_ner(tweet)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'B-organization',
  'index': 8,
  'score': 0.9995940327644348,
  'word': 'اتلتیکو'},
 {'entity': 'B-organization',
  'index': 13,
  'score': 0.9972712397575378,
  'word': 'بایرن'},
 {'entity': 'B-person',
  'index': 31,
  'score': 0.9915841221809387,
  'word': 'هاینکس'}]